### Overview

This is the first step for data preparation.

Window images, as well as reindex them 

Input: original CT images

Output: Images with abdominal windowing


In [27]:
%reset
%reload_ext autoreload
%autoreload 2
import numpy as np
import os
import glob
import SimpleITK as sitk

import sys

sys.path.insert(0, '../../dataloaders/')
# import niftiio as nio

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [28]:
# set up directories for images
IMG_FOLDER="img/"
SEG_FOLDER="label/"
OUT_FOLDER="./tmp_normalized/"

In [29]:
imgs = glob.glob(IMG_FOLDER + "/*.nii.gz")
print("imgs: ", imgs)
imgs = [ fid for fid in sorted(imgs) ]
segs = [ fid for fid in sorted(glob.glob(SEG_FOLDER + "/*.nii.gz")) ]

pids = [   pid.split("img0")[-1].split(".")[0] for pid in imgs]

imgs:  ['img\\img0001.nii.gz', 'img\\img0002.nii.gz', 'img\\img0003.nii.gz', 'img\\img0004.nii.gz', 'img\\img0005.nii.gz', 'img\\img0006.nii.gz', 'img\\img0007.nii.gz', 'img\\img0008.nii.gz', 'img\\img0009.nii.gz', 'img\\img0010.nii.gz', 'img\\img0021.nii.gz', 'img\\img0022.nii.gz', 'img\\img0023.nii.gz', 'img\\img0024.nii.gz', 'img\\img0025.nii.gz', 'img\\img0026.nii.gz', 'img\\img0027.nii.gz', 'img\\img0028.nii.gz', 'img\\img0029.nii.gz', 'img\\img0030.nii.gz', 'img\\img0031.nii.gz', 'img\\img0032.nii.gz', 'img\\img0033.nii.gz', 'img\\img0034.nii.gz', 'img\\img0035.nii.gz', 'img\\img0036.nii.gz', 'img\\img0037.nii.gz', 'img\\img0038.nii.gz', 'img\\img0039.nii.gz', 'img\\img0040.nii.gz']


In [30]:
# helper function
def copy_spacing_ori(src, dst):
    dst.SetSpacing(src.GetSpacing())
    dst.SetOrigin(src.GetOrigin())
    dst.SetDirection(src.GetDirection())
    return dst

In [31]:
import copy
scan_dir = OUT_FOLDER
LIR = -125
HIR = 275
os.makedirs(scan_dir, exist_ok = True)

reindex = 0
print("reindex: ", reindex)
print("imgs: ", imgs)
for img_fid, seg_fid, pid in zip(imgs, segs, pids):
    print("reindex: ", reindex)

    img_obj = sitk.ReadImage( img_fid )
    seg_obj = sitk.ReadImage( seg_fid )

    array = sitk.GetArrayFromImage(img_obj)

    array[array > HIR] = HIR
    array[array < LIR] = LIR
    
    array = (array - array.min()) / (array.max() - array.min()) * 255.0
    
    # then normalize this
    
    wined_img = sitk.GetImageFromArray(array)
    wined_img = copy_spacing_ori(img_obj, wined_img)
    
    out_img_fid = os.path.join( scan_dir, f'image_{str(reindex)}.nii.gz' )
    out_lb_fid  = os.path.join( scan_dir, f'label_{str(reindex)}.nii.gz' ) 
    
    # then save
    sitk.WriteImage(wined_img, out_img_fid, True) 
    sitk.WriteImage(seg_obj, out_lb_fid, True) 
    print("{} has been save".format(out_img_fid))
    print("{} has been save".format(out_lb_fid))
    reindex += 1


reindex:  0
imgs:  ['img\\img0001.nii.gz', 'img\\img0002.nii.gz', 'img\\img0003.nii.gz', 'img\\img0004.nii.gz', 'img\\img0005.nii.gz', 'img\\img0006.nii.gz', 'img\\img0007.nii.gz', 'img\\img0008.nii.gz', 'img\\img0009.nii.gz', 'img\\img0010.nii.gz', 'img\\img0021.nii.gz', 'img\\img0022.nii.gz', 'img\\img0023.nii.gz', 'img\\img0024.nii.gz', 'img\\img0025.nii.gz', 'img\\img0026.nii.gz', 'img\\img0027.nii.gz', 'img\\img0028.nii.gz', 'img\\img0029.nii.gz', 'img\\img0030.nii.gz', 'img\\img0031.nii.gz', 'img\\img0032.nii.gz', 'img\\img0033.nii.gz', 'img\\img0034.nii.gz', 'img\\img0035.nii.gz', 'img\\img0036.nii.gz', 'img\\img0037.nii.gz', 'img\\img0038.nii.gz', 'img\\img0039.nii.gz', 'img\\img0040.nii.gz']
reindex:  0
./tmp_normalized/image_0.nii.gz has been save
./tmp_normalized/label_0.nii.gz has been save
reindex:  1
./tmp_normalized/image_1.nii.gz has been save
./tmp_normalized/label_1.nii.gz has been save
reindex:  2
./tmp_normalized/image_2.nii.gz has been save
./tmp_normalized/label_2